# Data Reading

This notebook implements the data reading and manipulation for my thesis comparing different supervised machine learning techniques (logistic regression, random forests, support vector machines and neural networks). The main focus surrounds when each method results in better performance as measured by discrimination (AUC) and calibration (calibration plots). This project considers various factors including the number of features, number of datapoints and 'predictor strength'. 

In [1]:
#relevant packages
import pandas as pd
import numpy as np
import os
os.chdir(str(os.getcwd())+'\\Data\\2007')

## Data

The dataset used is the NHANES dataset (https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/) ran by the center for disease control. I used the 2007-2008 wave of NHANES such that over ten years had passed, as the dependent variable, ten-year mortality, requires this. This dataset was chosen as it is feature rich and datapoint rich, allowing for a thorough comparison of supervised ML methods.

In [2]:
#reading the various constituent pieces of the 2007-2008 NHANES, available on the website link above
demo = pd.read_sas('DEMO_E.XPT')
bm = pd.read_sas('BMX_E.XPT')
smk = pd.read_sas('SMQ_E.XPT')
alc = pd.read_sas('ALQ_E.XPT')
bp = pd.read_sas('BPX_E.XPT')
chol = pd.read_sas('TCHOL_E.XPT')
diab = pd.read_sas('DIQ_E.XPT')

In [3]:
#creates sel_demo, which takes the variables we are interested in
#from the demographic data file and recodes it in a more desireable format
sel_demo = demo.copy()[['SEQN', 'RIDAGEYR', 'RIAGENDR']]
sel_demo.rename(columns = {'SEQN': 'id',
                           'RIDAGEYR': 'age',
                           'RIAGENDR': 'sex'},
                           inplace = True)
sel_demo[['id', 'age', 'sex']] = sel_demo[['id', 'age', 'sex']].astype(int)
sel_demo.set_index('id', inplace = True)
sel_demo['sex'] = sel_demo['sex'].apply(lambda x: "Male" if x == 1 else "Female")

In [4]:
#creates sel_bm, which takes the variable we are interested in 
#from the body measurements data file and recodes it in a more desireable format
sel_bm = bm.copy()[['SEQN', 'BMXBMI']]
sel_bm.rename(columns = {'SEQN': 'id',
                         'BMXBMI': 'bmi'},
              inplace = True)
sel_bm[['id']] = sel_bm[['id']].astype(int)
sel_bm.set_index('id', inplace = True)

In [5]:
#creates sel_smk, which takes the variable we are interested in 
#from the smoking - cigarette use data file and recodes it in a more desireable format
#smoker is a yes if more than 100 lifetime cigarettes
sel_smk = smk.copy()[['SEQN', 'SMQ020']]
sel_smk.rename(columns = {'SEQN': 'id',
                          'SMQ020': 'smoke'},
               inplace = True)
sel_smk[['id']] = sel_smk[['id']].astype(int)
sel_smk.set_index('id', inplace = True)
sel_smk['smoke'] = sel_smk['smoke'].apply(lambda x: "Yes" if x == 1.0 else ("No" if x == 2.0 else np.NaN))

In [6]:
#creates sel_alc, which takes the variable we are interested in 
#from the alcohol use data file and recodes it in a more desireable format
#alcohol is a yes if consumed more than 12 alcoholic drinks per year.
sel_alc = alc.copy()[['SEQN', 'ALQ101']]
sel_alc.rename(columns = {'SEQN': 'id',
                          'ALQ101': 'alcohol'},
               inplace = True)
sel_alc[['id']] = sel_alc[['id']].astype(int)
sel_alc.set_index('id', inplace = True)
sel_alc['alcohol'] = sel_alc['alcohol'].apply(lambda x: "Yes" if x == 1.0 else ("No" if x == 2.0 else np.NaN))

In [7]:
#creates sel_bp, which takes the average of both systolic and diastolic blood pressures.
#measured in mmHg
sel_bp = bp.copy()[['SEQN', 'BPXSY1', 'BPXSY2', 'BPXSY3', 'BPXSY4', 'BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4']]
sel_bp['systolic'] = sel_bp[['BPXSY1', 'BPXSY2', 'BPXSY3', 'BPXSY4']].mean(axis = 1, skipna = True)
sel_bp['diastolic'] = sel_bp[['BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4']].mean(axis = 1, skipna = True)
sel_bp = sel_bp[['SEQN', 'systolic', 'diastolic']]
sel_bp.rename(columns = {'SEQN': 'id'},
               inplace = True)
sel_bp[['id']] = sel_bp[['id']].astype(int)
sel_bp.set_index('id', inplace = True)

In [8]:
#creates sel_chol, which takes the variable we are interested in 
#from the total cholesterol data file and recodes it in a more desirable format
#cholesterol in mmol/L
sel_chol = chol.copy()[['SEQN', 'LBDTCSI']]
sel_chol.rename(columns = {'SEQN': 'id',
                           'LBDTCSI': 'cholesterol'},
               inplace = True)
sel_chol[['id']] = sel_chol[['id']].astype(int)
sel_chol.set_index('id', inplace = True)

In [9]:
#creates sel_diab, which takes the variable we are interested in 
#from the diabetes data file and recodes it in a more desirable format
#diabetes a yes if patient told they had diabetes (borderline treated as missing)
sel_diab = diab.copy()[['SEQN', 'DIQ010']]
sel_diab.rename(columns = {'SEQN': 'id',
                           'DIQ010': 'diabetes'},
               inplace = True)
sel_diab[['id']] = sel_diab[['id']].astype(int)
sel_diab.set_index('id', inplace = True)
sel_diab['diabetes'] = sel_diab['diabetes'].apply(lambda x: "Yes" if x == 1.0 else ("No" if x == 2.0 else np.NaN))

In [10]:
#loading the mortality file (first ran through enclosed R script)
#and formatting it properly
lmf = pd.read_csv('lmf.csv')
sel_lmf = lmf.copy()[['seqn', 'eligstat', 'mortstat', 'permth_exm']]
sel_lmf.rename(columns = {'seqn': 'id',
                          'eligstat': 'eligible',
                          'mortstat': 'deceased',
                          'permth_exm': 'months'},
               inplace = True)
sel_lmf.set_index('id', inplace = True)
sel_lmf['eligible'] = sel_lmf['eligible'].apply(
    lambda x: "Eligible" if x == 1 else ("Ineligible" if x == 2 else np.NaN))
sel_lmf['mort10'] = np.where(sel_lmf['eligible']=='Eligible', 
                             np.logical_and(sel_lmf['deceased'] == 1, sel_lmf['months'] <= 120), 
                             np.NaN)
sel_lmf['mort10'] = sel_lmf['mort10'].apply(lambda x: "Yes" if x == 1.0 else ("No" if x == 0 else np.NaN))

sel_lmf = sel_lmf[['eligible','mort10']]

In [11]:
#merges all the data together
df = sel_demo.copy()
df = df.join(sel_bm, how = 'left')
df = df.join(sel_smk, how = 'left')
df = df.join(sel_alc, how = 'left')
df = df.join(sel_bp, how = 'left')
df = df.join(sel_chol, how = 'left')
df = df.join(sel_diab, how = 'left')
df = df.join(sel_lmf, how = 'left')

In [20]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
display(df.describe())
display(df.sex.value_counts())
display(df.smoke.value_counts())
display(df.alcohol.value_counts())
display(df.diabetes.value_counts())
display(df.mort10.value_counts())

,age,bmi,systolic,diastolic,cholesterol
count,10149.00,8861.00,7350.00,7350.00,7387.00
mean,32.97,25.71,119.97,66.63,4.83
std,25.48,7.61,18.90,14.19,1.09
min,0.00,12.50,74.00,0.00,2.02
25%,9.00,19.97,107.33,58.67,4.03
50%,29.00,25.16,116.67,67.33,4.68
75%,55.00,30.08,128.67,75.33,5.48
max,80.00,73.43,224.00,118.67,11.90


Male      5096
Female    5053
Name: sex, dtype: int64

No     3127
Yes    2802
Name: smoke, dtype: int64

Yes    3670
No     1557
Name: alcohol, dtype: int64

No     8774
Yes     777
Name: diabetes, dtype: int64

No     5362
Yes     857
Name: mort10, dtype: int64

In [12]:
comp_df = df.dropna(axis = 0, how = 'any') #making it complete case

In [19]:
#displaying summary
display(comp_df.describe())
display(comp_df.sex.value_counts())
display(comp_df.smoke.value_counts())
display(comp_df.alcohol.value_counts())
display(comp_df.diabetes.value_counts())
display(comp_df.mort10.value_counts())

,age,bmi,systolic,diastolic,cholesterol
count,4756.00,4756.00,4756.00,4756.00,4756.00
mean,50.56,28.94,124.13,69.94,5.10
std,17.73,6.55,18.63,13.02,1.08
min,20.00,14.20,80.67,0.00,2.35
25%,36.00,24.34,111.33,62.67,4.34
50%,50.00,28.05,120.67,70.67,5.02
75%,65.00,32.14,133.33,78.00,5.77
max,80.00,73.43,212.67,118.67,11.90


Female    2407
Male      2349
Name: sex, dtype: int64

No     2477
Yes    2279
Name: smoke, dtype: int64

Yes    3376
No     1380
Name: alcohol, dtype: int64

No     4157
Yes     599
Name: diabetes, dtype: int64

No     4098
Yes     658
Name: mort10, dtype: int64

In [14]:
#one hot encoding
bin_vars = comp_df[['sex', 'smoke', 'alcohol', 'diabetes', 'eligible', 'mort10']].copy() 
bin_vars = pd.get_dummies(bin_vars, drop_first = True)
bin_vars.rename(columns = {'sex_Male': 'male',
                           'smoke_Yes': 'smoker',
                           'alcohol_Yes': 'alcohol',
                           'diabetes_Yes': 'diabetes',
                           'mort10_Yes': 'deceased'},
                inplace = True)

In [15]:
comp_df = df[['age', 'bmi', 'systolic', 'diastolic', 'cholesterol']].copy().join(bin_vars, how = 'left')
comp_df.dropna(axis = 0, how = 'any', inplace = True)
comp_df
comp_df.to_csv('mortality.csv')